In [3]:
import pandas as pd
import plotly.express as px
from datetime import datetime, time, timedelta
import MetaTrader5 as mt5

In [26]:
mt5.initialize()

def get_rates(symbol, timeframe, start_dt, end_dt):
    rates = mt5.copy_rates_range(symbol, timeframe, start_dt, end_dt)
    rates_df = pd.DataFrame(rates)

    rates_df['time'] = pd.to_datetime(rates_df['time'], unit='s')
    rates_df['perc'] = (rates_df['close'] - rates_df['open']) / rates_df['open']
    rates_df['perc_cumsum'] = rates_df['perc'].cumsum()

    return rates_df[['time', 'open', 'high', 'low', 'close', 'perc', 'perc_cumsum']]

def get_ticks(symbol, start_dt, end_dt):
    ticks = mt5.copy_ticks_range(symbol, start_dt, end_dt, mt5.COPY_TICKS_ALL)
    ticks_df = pd.DataFrame(ticks)

    ticks_df['time'] = pd.to_datetime(ticks_df['time'], unit='s')

    return ticks_df[['time', 'bid', 'ask']]

In [27]:
symbol = 'DE40'

start = datetime.now() - timedelta(days=60)
end = datetime.now()

ticks = get_ticks(symbol, start, end)
ticks

,time,bid,ask
0,2023-08-09 17:50:36,15902.7,15903.2
1,2023-08-09 17:50:37,15903.2,15903.7
2,2023-08-09 17:50:37,15902.7,15903.2
3,2023-08-09 17:50:38,15903.2,15903.7
4,2023-08-09 17:50:39,15902.7,15903.2
...,...,...,...
1597011,2023-10-06 22:59:48,15278.5,15279.7
1597012,2023-10-06 22:59:52,15277.0,15278.2
1597013,2023-10-06 22:59:55,15277.5,15278.7
1597014,2023-10-06 22:59:59,15279.0,15280.2


In [28]:
ticks['date'] = ticks['time'].dt.date
ticks

,time,bid,ask,date
0,2023-08-09 17:50:36,15902.7,15903.2,2023-08-09
1,2023-08-09 17:50:37,15903.2,15903.7,2023-08-09
2,2023-08-09 17:50:37,15902.7,15903.2,2023-08-09
3,2023-08-09 17:50:38,15903.2,15903.7,2023-08-09
4,2023-08-09 17:50:39,15902.7,15903.2,2023-08-09
...,...,...,...,...
1597011,2023-10-06 22:59:48,15278.5,15279.7,2023-10-06
1597012,2023-10-06 22:59:52,15277.0,15278.2,2023-10-06
1597013,2023-10-06 22:59:55,15277.5,15278.7,2023-10-06
1597014,2023-10-06 22:59:59,15279.0,15280.2,2023-10-06


In [29]:
ticks2 = ticks.groupby('date', as_index=False).agg(
    high=('ask', 'max'),
    low=('bid', 'min')
)
ticks2['prev_high'] = ticks2['high'].shift(1)
ticks2['prev_low'] = ticks2['low'].shift(1)

ticks2 = ticks2[1:-1]
ticks2

,date,high,low,prev_high,prev_low
1,2023-08-10,16062.6,15874.6,15931.1,15829.2
2,2023-08-11,15976.7,15799.1,16062.6,15874.6
3,2023-08-14,15941.9,15751.6,15976.7,15799.1
4,2023-08-15,15965.2,15699.8,15941.9,15751.6
5,2023-08-16,15817.8,15669.2,15965.2,15699.8
6,2023-08-17,15796.9,15593.9,15817.8,15669.2
7,2023-08-18,15665.7,15468.1,15796.9,15593.9
8,2023-08-21,15712.3,15562.5,15665.7,15468.1
9,2023-08-22,15798.9,15616.6,15712.3,15562.5
10,2023-08-23,15827.7,15669.7,15798.9,15616.6


In [33]:
ticks3 = ticks.merge(ticks2[['date', 'prev_high', 'prev_low']])
ticks3['break_high'] = ticks3.apply(lambda x: 1 if x['bid'] > x['prev_high'] else 0, axis=1)
ticks3['break_low'] = ticks3.apply(lambda x: 1 if x['ask'] < x['prev_low'] else 0, axis=1)
ticks3

,time,bid,ask,date,prev_high,prev_low,break_high,break_low
0,2023-08-10 01:02:15,15901.5,15907.5,2023-08-10,15931.1,15829.2,0,0
1,2023-08-10 01:02:21,15901.3,15907.3,2023-08-10,15931.1,15829.2,0,0
2,2023-08-10 01:02:23,15901.1,15907.1,2023-08-10,15931.1,15829.2,0,0
3,2023-08-10 01:02:26,15901.5,15907.5,2023-08-10,15931.1,15829.2,0,0
4,2023-08-10 01:02:31,15901.3,15907.3,2023-08-10,15931.1,15829.2,0,0
...,...,...,...,...,...,...,...,...
1543055,2023-10-05 23:58:22,15102.1,15108.1,2023-10-05,15176.8,14943.8,0,0
1543056,2023-10-05 23:58:31,15101.9,15107.9,2023-10-05,15176.8,14943.8,0,0
1543057,2023-10-05 23:58:44,15101.6,15107.6,2023-10-05,15176.8,14943.8,0,0
1543058,2023-10-05 23:58:51,15101.4,15107.4,2023-10-05,15176.8,14943.8,0,0


In [ ]:
ticks3['first_break'] = ticks3